In [ ]:
import geopandas as gpd
import networkx as nx
from shapely.ops import unary_union
from shapely.validation import make_valid
import pandas as pd
import logging
from tqdm import tqdm

# 配置日志输出到控制台
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 输入和输出路径
input_shp = r"C:\Users\Runker\Desktop\shp\test.shp"  # 替换为您的实际输入路径
output_shp = r"C:\Users\Runker\Desktop\shp\testsss.shp"  # 替换为您的实际输出路径

# 读取数据
gdf = gpd.read_file(input_shp)
original_crs = gdf.crs  # 保存原始坐标系

# 修复无效几何
gdf['geometry'] = gdf['geometry'].apply(lambda geom: make_valid(geom) if not geom.is_valid else geom)

# 计算初始总面积
original_area = gdf.geometry.area.sum()
logging.info(f"原始总面积: {original_area:.2f} 平方米")

# 如果没有面积列，计算面积
if 'area' not in gdf.columns:
    gdf['area'] = gdf.geometry.area

# 按DLMC分组
grouped = gdf.groupby('DLMC')
merged_gdf_list = []

# 处理每个DLMC类型
for dlmc, group in tqdm(grouped, desc="处理DLMC类型"):
    logging.info(f"正在处理DLMC类型: {dlmc}")
    
    # 根据DLMC类型设置不同的面积阈值
    if dlmc in ['水田', '旱地', '水浇地']:
        threshold = 100
    else:
        threshold = 500
    
    small_parcels = group[group['area'] < threshold]
    large_parcels = group[group['area'] >= threshold].copy()  # 创建副本
    
    if small_parcels.empty:
        logging.info(f"DLMC类型 {dlmc} 没有小图斑需要处理")
        merged_gdf_list.append(large_parcels)
        continue
    
    # 用于跟踪已合并的小图斑
    merged_small_indices = set()
    
    # 构建邻接图（仅小图斑之间）
    G = nx.Graph()
    for idx, geom in tqdm(small_parcels.iterrows(), total=len(small_parcels), desc=f"构建邻接图 - {dlmc}"):
        G.add_node(idx)
        possible_matches_idx = list(small_parcels.sindex.query(geom.geometry, predicate='touches'))
        neighbors = small_parcels.index[possible_matches_idx]
        for neighbor in neighbors:
            if neighbor != idx:  # 避免自连接
                G.add_edge(idx, neighbor)
    
    # 识别连通分量
    components = list(nx.connected_components(G))
    logging.info(f"DLMC类型 {dlmc} 有 {len(components)} 个连通分量")
    
    # 合并连通分量中的小图斑
    for component in tqdm(components, desc=f"合并连通分量 - {dlmc}"):
        if len(component) == 1:  # 单节点跳到孤立小图斑处理
            continue
        geoms = small_parcels.loc[list(component), 'geometry']
        merged_geom = unary_union(geoms)
        
        # 检查面积变化
        original_area_sum = sum(geoms.area)
        merged_area = merged_geom.area
        if abs(merged_area - original_area_sum) > 10:
            logging.warning(f"小图斑连通分量面积变化超过阈值: {original_area_sum:.2f} -> {merged_area:.2f}")
            continue
        
        # 记录已合并的小图斑索引
        merged_small_indices.update(component)
        
        new_row = small_parcels.loc[list(component)[0]].copy()
        new_row['geometry'] = merged_geom
        new_row['area'] = merged_area
        merged_gdf_list.append(pd.DataFrame([new_row]))
    
    # 处理孤立小图斑（无其他小图斑邻居，但有大图斑邻居）
    isolated_nodes = [node for node in G.nodes() if G.degree(node) == 0]
    if isolated_nodes:
        logging.info(f"DLMC类型 {dlmc} 有 {len(isolated_nodes)} 个孤立小图斑，开始与大图斑合并")
        for idx in tqdm(isolated_nodes, desc=f"处理孤立小图斑 - {dlmc}"):
            small_geom = small_parcels.loc[idx, 'geometry']
            # 查找相邻的大图斑
            large_neighbors_idx = list(large_parcels.sindex.query(small_geom, predicate='touches'))
            large_neighbors = large_parcels.index[large_neighbors_idx]
            
            if large_neighbors.empty:
                # 无大图斑邻居，保留孤立小图斑
                if idx not in merged_small_indices:
                    merged_gdf_list.append(pd.DataFrame([small_parcels.loc[idx]]))
            else:
                # 选择面积最大的相邻大图斑
                target_idx = large_parcels.loc[large_neighbors, 'area'].idxmax()
                target_geom = large_parcels.loc[target_idx, 'geometry']
                merged_geom = unary_union([small_geom, target_geom])
                
                # 检查面积变化
                original_area_sum = small_geom.area + target_geom.area
                merged_area = merged_geom.area
                if abs(merged_area - original_area_sum) > 10:
                    logging.warning(f"孤立小图斑与大图斑合并面积变化超过阈值: {original_area_sum:.2f} -> {merged_area:.2f}")
                    continue
                
                # 更新大图斑并标记小图斑为已合并
                large_parcels.loc[target_idx, 'geometry'] = merged_geom
                large_parcels.loc[target_idx, 'area'] = merged_area
                merged_small_indices.add(idx)
    
    # 仅保留未合并的小图斑和大图斑
    remaining_small_parcels = small_parcels.drop(list(merged_small_indices), errors='ignore')
    if not remaining_small_parcels.empty:
        merged_gdf_list.append(remaining_small_parcels)
    merged_gdf_list.append(large_parcels)

# 合并所有结果
merged_gdf = gpd.GeoDataFrame(pd.concat(merged_gdf_list, ignore_index=True))

# 检查最终面积
final_area = merged_gdf.geometry.area.sum()
logging.info(f"处理后总面积: {final_area:.2f} 平方米")
logging.info(f"面积变化: {final_area - original_area:.2f} 平方米")

# 处理列名，确保不超过10个字符（Shapefile限制）
merged_gdf.columns = [str(col)[:10] for col in merged_gdf.columns]

# 保存为Shapefile，保留原始CRS
merged_gdf.to_file(output_shp, driver='ESRI Shapefile', crs=original_crs, encoding='utf-8')
logging.info(f"结果已保存到: {output_shp}")

# 清理内存
del gdf, grouped, merged_gdf_list

In [ ]:
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
# 支持中文
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
from shapely.geometry import Polygon
from tqdm import tqdm

# 创建示例数据
polygons = [
    Polygon([(0, 0), (2, 0), (2, 2), (0, 2)]),  # 图斑 0
    Polygon([(2, 0), (4, 0), (4, 2), (2, 2)]),  # 图斑 1
    Polygon([(2, 2), (4, 2), (4, 4), (2, 4)]),  # 图斑 2
    Polygon([(5, 0), (7, 0), (7, 2), (5, 2)]),  # 图斑 3
    Polygon([(7, 0), (9, 0), (9, 2), (7, 2)])   # 图斑 4
]

small_parcels = gpd.GeoDataFrame({
    'geometry': polygons,
    'area': [4, 4, 4, 4, 4]  # 假设面积均为 4，小于阈值
}, crs="EPSG:4326")

# 构建邻接图
G = nx.Graph()
dlmc = "示例类型"  # 假设的DLMC类型
for idx, geom in tqdm(small_parcels.iterrows(), total=len(small_parcels), desc=f"构建邻接图 - {dlmc}"):
    G.add_node(idx)
    neighbors = small_parcels[small_parcels.geometry.touches(geom.geometry)].index
    for neighbor in neighbors:
        G.add_edge(idx, neighbor)

# 识别连通分量
components = list(nx.connected_components(G))
print(f"连通分量: {components}")

# 可视化邻接图
pos = {i: (poly.centroid.x, poly.centroid.y) for i, poly in enumerate(polygons)}  # 使用几何中心作为节点位置
plt.figure(figsize=(8, 6))
nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=500, font_size=12, edge_color='gray')
plt.title("邻接图示例")
plt.show()

# 可视化原始几何和连通分量
fig, ax = plt.subplots(figsize=(8, 6))
small_parcels.plot(ax=ax, facecolor='none', edgecolor='blue', linewidth=1)
for i, component in enumerate(components):
    centroids = [polygons[idx].centroid for idx in component]
    x = [c.x for c in centroids]
    y = [c.y for c in centroids]
    ax.plot(x, y, marker='o', linestyle='-', label=f"连通分量 {i+1}")
ax.legend()
plt.title("图斑和连通分量")
plt.show()